In [ ]:
import os
import pandas as pd
import mygene

# Initialize MyGene client
mg = mygene.MyGeneInfo()

# Set working directory
os.chdir("C:/Users/druschel/Documents/Python/Gene Homolog")
print("Working directory set to:", os.getcwd())



In [2]:
# Define species and taxonomic ID mapping
species_map = {"Mouse": 10090, "Human": 9606, "Rat": 10116}

# Set starting and target species
starting_species = "Rat"  # Starting species (e.g., "Rat")
target_species = "Mouse"  # Target species (e.g., "Mouse")

# Get taxonomic IDs for starting and target species
starting_species_id = species_map[starting_species]
target_species_id = species_map[target_species]

print(f"Starting Species: {starting_species} (ID: {starting_species_id})")
print(f"Target Species: {target_species} (ID: {target_species_id})")


Starting Species: Rat (ID: 10116)
Target Species: Mouse (ID: 10090)


In [3]:
# Load the input Excel file
file_path = "Yulia all 17k important FPKM.xlsx"  # Replace with your actual file name
data = pd.read_excel(file_path)

# Ensure the file contains a "gene_id" column
if "gene_id" not in data.columns:
    raise ValueError("The input file must contain a 'gene_id' column.")

# Preview the first few rows of the DataFrame
data.head()


,gene_id,D2_No1_fpkm,D2_No2_fpkm,D2_No3_fpkm,D2_No5_fpkm,D4_No1_fpkm,D4_No2_fpkm,D4_No3_fpkm,D4_No4_fpkm,D4_No5_fpkm
0,CCDC152,10.668435,6.954662,12.200111,10.010649,102.551143,55.588696,93.171158,75.309351,73.353697
1,SORBS3,9.915534,6.849236,6.577316,6.668627,28.896706,29.515292,33.654237,34.578308,28.114348
2,CNDP1,9.138440,7.771208,8.767425,4.258629,47.758615,48.887337,74.970720,42.565679,49.474445
3,PLTP,111.935421,53.245470,51.605032,64.655446,928.713303,1192.562223,1522.364417,584.787829,700.721307
4,PPFIBP2,2.181392,2.127358,1.680455,1.501055,7.054794,6.236110,6.302764,5.821146,6.417733


In [4]:
# Extract genes from the input data
genes = data["gene_id"].tolist()

# Query homolog data from MyGene.info
results = mg.querymany(genes, scopes="symbol", fields="homologene", species=starting_species_id, verbose=True)

# Preview the first few query results
results[:5]


833 input query terms found dup hits:	[('PLTP', 2), ('CD44', 2), ('RAB42', 2), ('PHYH', 2), ('MK1', 2), ('NBL1', 2), ('CASD1', 2), ('DGLUC
3045 input query terms found no hit:	['METTL7A', 'AABR07035470.1', 'AABR07044631.2', 'AABR07026294.1', 'AABR07071838.1', 'LOC24906', 'AAB


[{'query': 'CCDC152',
  '_id': '499536',
  '_score': 12.877283,
  'homologene': {'genes': [[7955, 797096],
    [8364, 100491771],
    [9031, 101747341],
    [9598, 461898],
    [9606, 100129792],
    [9615, 100687996],
    [9913, 767877],
    [10090, 100039139],
    [10116, 499536]],
   'id': 79247}},
 {'query': 'SORBS3',
  '_id': '282843',
  '_score': 12.570777,
  'homologene': {'genes': [[7955, 541478],
    [9544, 716198],
    [9598, 472712],
    [9606, 10174],
    [9615, 477383],
    [9913, 507914],
    [10090, 20410],
    [10116, 282843]],
   'id': 4218}},
 {'query': 'CNDP1',
  '_id': '307212',
  '_score': 12.726513,
  'homologene': {'genes': [[8364, 100135225],
    [9031, 421012],
    [9544, 695195],
    [9598, 455474],
    [9606, 84735],
    [9615, 476165],
    [9913, 614200],
    [10090, 338403],
    [10116, 307212]],
   'id': 57178}},
 {'query': 'PLTP', '_id': 'ENSRNOG00000016488', '_score': 12.598684},
 {'query': 'PLTP',
  '_id': '296371',
  '_score': 12.598684,
  'homologene'

In [5]:

# Step 1: Extract homolog gene IDs - takes the highest confidence homolog but does NOT filter based on confidence
def process_homolog_ids_highest_score(results, target_species_id):
    """
    Process MyGene query results to extract homolog IDs for a target species,
    selecting the homolog with the highest score if multiple are found.

    Args:
        results (list): List of query results from MyGene.
        target_species_id (int): Taxonomic ID of the target species.

    Returns:
        dict: Dictionary mapping input gene names to homolog IDs.
    """
    result_dict = {}
    
    for res in results:
        query = res.get("query")  # The queried gene name
        best_homolog = ""  # Default to blank if no match is found
        best_score = float('-inf')  # Track the highest score
        
        # Check if "homologene" exists and contains valid data
        if "homologene" in res and "genes" in res["homologene"]:
            homolog_list = res["homologene"]["genes"]
            
            for entry in homolog_list:
                species_id, homolog_id = entry
                # Check if the entry matches the target species
                if species_id == target_species_id:
                    # Get the `_score` for the result (default to 0 if missing)
                    score = res.get("_score", 0)
                    # Update the best homolog if this one has a higher score
                    if score > best_score:
                        best_score = score
                        best_homolog = str(homolog_id)
        
        # Add the best match to the result dictionary
        result_dict[query] = best_homolog

    return result_dict

# Process query results to extract homolog IDs with the highest confidence
homolog_id_dict = process_homolog_ids_highest_score(results, target_species_id)



In [6]:
# Step 2: Query MyGene.info for gene names
def fetch_gene_names(homolog_ids):
    """
    Fetch gene names (symbols) for homolog IDs using MyGene.info.

    Args:
        homolog_ids (dict): Dictionary mapping gene names to homolog IDs.
    
    Returns:
        dict: Dictionary mapping homolog IDs to gene names.
    """
    gene_name_dict = {}
    # Filter out "No Match" and empty homolog IDs
    valid_ids = [homolog_id for homolog_id in homolog_ids.values() if homolog_id != "No Match"]
    
    # Query MyGene.info with homolog IDs
    homolog_name_results = mg.querymany(valid_ids, scopes="entrezgene", fields="symbol", species=target_species_id, verbose=True)
    
    for res in homolog_name_results:
        gene_id = res.get("query")
        gene_name = res.get("symbol", "No Name Found")  # Default to "No Name Found" if symbol is missing
        gene_name_dict[gene_id] = gene_name

    return gene_name_dict

# Fetch gene names for homolog IDs
homolog_name_dict = fetch_gene_names(homolog_id_dict)


19 input query terms found dup hits:	[('15078', 2), ('17067', 2), ('56554', 2), ('110460', 2), ('12313', 2), ('269261', 2), ('100503605',


In [7]:
# Map homolog names to input data
data["New_Homolog"] = [homolog_name_dict.get(homolog_id_dict.get(gene, ""), "No Match") for gene in data["gene_id"]]

# Preview the updated DataFrame
data.head()


,gene_id,D2_No1_fpkm,D2_No2_fpkm,D2_No3_fpkm,D2_No5_fpkm,D4_No1_fpkm,D4_No2_fpkm,D4_No3_fpkm,D4_No4_fpkm,D4_No5_fpkm,New_Homolog
0,CCDC152,10.668435,6.954662,12.200111,10.010649,102.551143,55.588696,93.171158,75.309351,73.353697,Ccdc152
1,SORBS3,9.915534,6.849236,6.577316,6.668627,28.896706,29.515292,33.654237,34.578308,28.114348,Sorbs3
2,CNDP1,9.138440,7.771208,8.767425,4.258629,47.758615,48.887337,74.970720,42.565679,49.474445,Cndp1
3,PLTP,111.935421,53.245470,51.605032,64.655446,928.713303,1192.562223,1522.364417,584.787829,700.721307,Pltp
4,PPFIBP2,2.181392,2.127358,1.680455,1.501055,7.054794,6.236110,6.302764,5.821146,6.417733,Ppfibp2


In [8]:
# Ensure all entries in the 'gene_id' column are strings
data["gene_id"] = data["gene_id"].astype(str)

# Function to match the case style of the input
def match_case(input_name, output_name):
    if input_name.isupper():  # If the input is all uppercase
        return output_name.upper()
    elif input_name.istitle():  # If the input is title case
        return output_name.title()
    else:  # Default to lowercase if input has mixed or lowercase
        return output_name.lower()

# Map homolog names to input data while preserving case
data["New_Homolog"] = [
    match_case(gene, homolog_name_dict.get(homolog_id_dict.get(gene, ""), ""))
    for gene in data["gene_id"]
]

# Preview the updated DataFrame
data.head()



,gene_id,D2_No1_fpkm,D2_No2_fpkm,D2_No3_fpkm,D2_No5_fpkm,D4_No1_fpkm,D4_No2_fpkm,D4_No3_fpkm,D4_No4_fpkm,D4_No5_fpkm,New_Homolog
0,CCDC152,10.668435,6.954662,12.200111,10.010649,102.551143,55.588696,93.171158,75.309351,73.353697,CCDC152
1,SORBS3,9.915534,6.849236,6.577316,6.668627,28.896706,29.515292,33.654237,34.578308,28.114348,SORBS3
2,CNDP1,9.138440,7.771208,8.767425,4.258629,47.758615,48.887337,74.970720,42.565679,49.474445,CNDP1
3,PLTP,111.935421,53.245470,51.605032,64.655446,928.713303,1192.562223,1522.364417,584.787829,700.721307,PLTP
4,PPFIBP2,2.181392,2.127358,1.680455,1.501055,7.054794,6.236110,6.302764,5.821146,6.417733,PPFIBP2


In [9]:
## Save the updated data to an Excel file
#output_excel_file = "output_genes_with_homologs.xlsx"  # Replace with your desired file name
#data.to_excel(output_excel_file, index=False)
#print(f"Processed Excel file saved to: {output_excel_file}")


# Save all results (including blanks) to a CSV file
output_csv_all = "output_genes_all.csv"
data.to_csv(output_csv_all, index=False)
print(f"All results saved to: {output_csv_all}")

# Filter to keep only matched genes (non-blank New_Homolog)
matched_data = data[data["New_Homolog"] != ""]

# Save matched results to a separate CSV file
output_csv_matched = "output_genes_matched.csv"
matched_data.to_csv(output_csv_matched, index=False)
print(f"Matched results saved to: {output_csv_matched}")

# Calculate and print match rate
total_genes = len(data)
matched_genes = len(matched_data)
match_rate = (matched_genes / total_genes) * 100
print(f"Match rate: {matched_genes}/{total_genes} genes matched ({match_rate:.2f}%)")

All results saved to: output_genes_all.csv
Matched results saved to: output_genes_matched.csv
Match rate: 12833/17192 genes matched (74.65%)
